# Chapter 4

<a target="_blank" href="https://colab.research.google.com/github/wandb/edu/blob/e98981ef9d934b10a0c6b4855d8eb6bfc7f56f1a/rag-advanced/notebooks/Chapter04.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

<!--- @wandbcode{rag-advance-chapter-04} -->

## Query Enhancement

Improving the quality of data helps with improving the quality of generated response. Another way is to improve the quality of the query seen by the LLM.

We cannot ask the user to provide the query in the best way possible. Many a times the user is not very sure of the query to be asked. Query enhancement as the name suggests, is an intermediate step that uses LLM to enhance the quality of the query. The enhancement can be - 
- making the query gramatically correct
- breaking down a complex query into relevant sub-queries
- extract the intent of the query (this can be passed for formatted answer in case of nefarious queries)
- if you have a chat history, augment the query with past queries and generated answers/retrieved contexts.
- extract keywords (can be about your product or anything related to your application) and pass it along with the query to your LLM

One can imagine many different ways to enhance the quality of the query or extract meaningful stuff from a query.

To begin, execute the following cell to clone the repository and install dependencies:

In [ ]:
!git clone https://github.com/wandb/edu.git
%cd edu
!git checkout rag-irl

%cd rag-advanced
!pip install -qqq -U uv
!uv pip install --system -r requirements.txt
%cd notebooks

With the setup complete, we can now proceed with the chapter content.

Initial steps:
1. Log in to Weights & Biases (W&B)
2. Configure environment variables for API access

To obtain your Cohere API key, visit the [Cohere API dashboard](https://dashboard.cohere.com/api-keys).

In [ ]:
import getpass
import os

import wandb

os.environ["COHERE_API_KEY"] = getpass.getpass("Please enter your COHERE_API_KEY")
wandb.login()

In [1]:
%load_ext autoreload
%autoreload 2

import asyncio
import cohere
import weave

In [2]:
WANDB_ENTITY = "rag-course"
WANDB_PROJECT = "dev"

weave_client = weave.init(f"{WANDB_ENTITY}/{WANDB_PROJECT}")

weave version 0.51.6 is available!  To upgrade, please run:
 $ pip install weave --upgrade
Logged in as Weights & Biases user: parambharat.
View Weave data at https://wandb.ai/rag-course/dev/weave


We will download the chunked data from chapter 3. This chunking was done using semantic chunking strategy.

In [3]:
# Reload the data from Chapter 3
chunked_data = weave.ref('chunked_data:v0').get()

chunked_data.rows[:2]

[WeaveDict({'content': '--- description: Log and visualize data without a W&B account displayed_sidebar: default --- # Anonymous Mode Are you publishing code that you want anyone to be able to run easily? Use Anonymous Mode to let someone run your code, see a W&B dashboard, and visualize results without needing to create a W&B account first. Allow results to be logged in Anonymous Mode with `wandb.init(`**`anonymous="allow"`**`)` :::info **Publishing a paper?** Please [cite W&B](https://docs.wandb.ai/company/academics#bibtex-citation), and if you have questions about how to make your code accessible while using W&B, reach out to us at support@wandb.com. ::: ### How does someone without an account see results? If someone runs your script and you have to set `anonymous="allow"`: 1. **Auto-create temporary account:** W&B checks for an account that\'s already signed in. If there\'s no account, we automatically create a new anonymous account and save that API key for the session. 2. **Log r

In our usecase we will use this query enhancement stage to -
- identify the language of the query (our documentation in in English, Japanese and Korean and we want to answer in the language of the query)
- indentify the intent of the query (a user might ask something that is not related to our documentation)
- generate sub-queries (break down the main query into smaller queries) for retrieving more contexts for our LLM.

These additional informations will be used to inform the response generator and improve the retrieval process.

In [4]:
from scripts.query_enhancer import QueryEnhancer
from scripts.utils import display_source

query_enhancer = QueryEnhancer()

In [5]:
response = await query_enhancer.predict("How do I log images in lightning with wandb?")

Look at the response below:

- we identified the query to be in English.
- We derived few sub-queries that make sense.
- We classified the intent based on our intent classification "prompt/guides"

In [6]:
response

{'query': 'How do I log images in lightning with wandb?',
 'language': 'en',
 'search_queries': ['How to log images in lightning with wandb?',
  'How to log images in lightning?',
  'Log images in wandb',
  'What is wandb?',
  'What is lightning?'],
 'intents': [{'intent': 'integrations',
   'reason': 'The user is asking about logging images in Lightning, which is a specific integration with Weights & Biases. They want to know how to utilize this integration effectively.'}]}

Our retriever will remain the same. Yes we have 5 sub-queries that we want to retrieve for but we can do so one by one. 

Let us use our BM25 based retriever from Chapter 2 and index our chunked data.

In [7]:
from scripts.retriever import BM25Retriever
retriever = BM25Retriever()
retriever.index_data(chunked_data.rows)

Split strings:   0%|          | 0/1073 [00:00<?, ?it/s]

Since we have more information extracted from our query - like the language and the intent of the query, we write `QueryEnhanedResponseGenerator` whihc uses a new system prompt augmented with language and intent information.

Look at line 24.

In [8]:
from scripts.response_generator import QueryEnhanedResponseGenerator
display_source(QueryEnhanedResponseGenerator)

   1 class QueryEnhanedResponseGenerator(weave.Model):                                                             
   2     model: str                                                                                                
   3     prompt: str                                                                                               
   4     client: cohere.AsyncClient = None                                                                         
   5                                                                                                               
   6     def __init__(self, **kwargs):                                                                             
   7         super().__init__(**kwargs)                                                                            
   8         self.client = cohere.AsyncClientV2(api_key=os.environ["COHERE_API_KEY"])                              
   9                                                                                                               
  10     @weave.op()                                                                                               
  11     def generate_context(self, context: List[Dict[str, any]]) -> List[Dict[str, any]]:                        
  12         contexts = [                                                                                          
  13             {"source": item["source"], "text": item["text"]} for item in context                              
  14         ]                                                                                                     
  15         return contexts                                                                                       
  16                                                                                                               
  17     def create_messages(                                                                                      
  18         self,                                                                                                 
  19         query: str,                                                                                           
  20         context: List[Dict[str, any]],                                                                        
  21         language: str,                                                                                        
  22         intents: List[str],                                                                                   
  23     ):                                                                                                        
  24         documents = self.generate_context(context)                                                            
  25                                                                                                               
  26         messages = [                                                                                          
  27             {                                                                                                 
  28                 "role": "system",                                                                             
  29                 "content": self.prompt.format(language=language, intents=intents),                            
  30             [38;2;248;248;

The `QueryEnhancedRAGPipeline` runs through different `search_queries` or sub-queries and retrieve the chunks. It also deduplicate the chunks so that we don't end up sending the same chunk twice.

Note line 23-27. We check if the extracted intent is not in a list of intents to avoid. If that's the case, we do not do retrieval and can return a formatted answer like - "This query is not related to Weights and Biases. Can you please ask again?"

In [9]:
from scripts.rag_pipeline import QueryEnhancedRAGPipeline
display_source(QueryEnhancedRAGPipeline)

   1 class QueryEnhancedRAGPipeline(weave.Model):                                                                  
   2     query_enhancer: weave.Model = None                                                                        
   3     retriever: weave.Model = None                                                                             
   4     response_generator: weave.Model = None                                                                    
   5     top_k: int = 5                                                                                            
   6                                                                                                               
   7     @weave.op()                                                                                               
   8     async def predict(self, query: str):                                                                      
   9         # enhance the query                                                                                   
  10         enhanced_query = await self.query_enhancer.predict(query)                                             
  11         user_query = enhanced_query["query"]                                                                  
  12                                                                                                               
  13         avoid_intents = [                                                                                     
  14             "unrelated",                                                                                      
  15             "needs_more_info",                                                                                
  16             "opinion_request",                                                                                
  17             "nefarious_query",                                                                                
  18             "other",                                                                                          
  19         ]                                                                                                     
  20                                                                                                               
  21         avoid_retrieval = False                                                                               
  22                                                                                                               
  23         intents = enhanced_query["intents"]                                                                   
  24         for intent in intents:                                                                                
  25             if intent["intent"] in avoid_intents:                                                             
  26                 avoid_retrieval = True                                                                        
  27                 break                                                                                         
  28                                                                                                               
  29         language = enhanced_query["language"]                                                                 
  30                                                                                                               
  31         # run retrieval for multiple queries                                                                  
  32         contexts = []                                                                                         
  33         if not avoid_retrieval:                                                                               
  34             retriever_queries = enhanced_query["search_queries"]                                              
  35             contexts.append(self.retriever.predict(

Let us initialize the response generator and our RAG pipeline and run in on one query.

In [10]:
# lets add the new prompt
QUERY_ENHANCED_PROMPT = open("prompts/query_enhanced_system.txt").read()

response_generator = QueryEnhanedResponseGenerator(
    model="command-r", prompt=QUERY_ENHANCED_PROMPT, client=cohere.AsyncClient()
)

In [11]:
query_enhanced_rag_pipeline = QueryEnhancedRAGPipeline(
    query_enhancer=query_enhancer,
    retriever=retriever,
    response_generator=response_generator,
    top_k=5,
)

response = await query_enhanced_rag_pipeline.predict("How do I log images in lightning with wandb?")
from IPython.display import Markdown
Markdown(response)

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

You can use PyTorch Lightning's WandbLogger to log images in lightning with Weights & Biases (W&B). Below is an example code snippet demonstrating the usage:
```python
import torch
import wandb
import lightning.pytorch as pl
from lightning.pytorch.loggers import WandbLogger

wandb_logger = WandbLogger()
trainer = pl.Trainer(logger=wandb_logger)

# Option 1: Log images with `WandbLogger.log_image`
wandb_logger.log_image(
    key="sample_images",
    images=[image for image in images],
    caption=captions
)

# Option 2: Log images and predictions as a W&B Table
columns = ["image", "ground truth", "prediction"]
data = [
    [wandb.Image(x_i), y_i, y_pred] 
    for x_i, y_i, y_pred 
    in zip(images, ground_truths, predictions)
]
wandb_logger.log_table(key="sample_table", columns=columns, data=data)
```

In the above code, `WandbLogger()` is instantiated to create an object `wandb_logger`, which is then passed to `Trainer()` to create an object `trainer`. Then, you can use the `wandb_logger` object to log images in two different ways. 

In Option 1, call the `log_image` method on the `wandb_logger` object and specify the key of the image in the `key` parameter, the list of images to be logged in the `images` parameter, and the captions of the images in the `caption` parameter.

In Option 2, first define the columns of the Table as a list in the `columns` variable and create the data to be logged as a list of lists in the `data` variable. Then, call the `log_table` method on the `wandb_logger` object, passing the key of the table in the `key` parameter, the column definitions in the `columns` parameter, and the table data in the `data` parameter.

You can also use W&B's `wandb.log()` function along with PyTorch Lightning's Callbacks system to log images. Here's an example:
```python
<co: 1

## Evaluate and Compare

In [12]:
eval_dataset = weave.ref(
    "weave:///rag-course/dev/object/Dataset:Qj4IFICc2EbdXu5A5UuhkPiWgxM1GvJMIvXEyv1DYnM"
).get()

print(eval_dataset.rows[:2])

[WeaveDict({'question': 'How can I access the run object from the Lightning WandBLogger function?', 'answer': "In PyTorch Lightning, the `WandbLogger` is used to log metrics, model weights, and other data to Weights & Biases during training. To access the `wandb.Run` object from within a `LightningModule` when using `WandbLogger`, you can use the `Trainer.logger.experiment` attribute. This attribute provides direct access to the underlying `wandb.Run` object, allowing you to interact with the Weights & Biases API directly.\n\nHere's how you can access the `wandb.Run` object using `WandbLogger` in PyTorch Lightning:\n\n```python\nfrom pytorch_lightning import Trainer, LightningModule\nfrom pytorch_lightning.loggers import WandbLogger\n\nclass MyModel(LightningModule):\n    def training_step(self, batch, batch_idx):\n        # Your training logic here\n        loss = ...\n\n        # Log metrics\n        self.log('train_loss', loss)\n\n        # Access the wandb.Run object\n        run =

In [13]:
# Let also initialize the baseline RAG pipeline from chapter 3

from scripts.rag_pipeline import SimpleRAGPipeline
from scripts.response_generator import SimpleResponseGenerator

INITIAL_PROMPT = open("prompts/initial_system.txt", "r").read()
response_generator = SimpleResponseGenerator(model="command-r", prompt=INITIAL_PROMPT)
simple_rag_pipeline = SimpleRAGPipeline(
    retriever=retriever, response_generator=response_generator, top_k=5
)

In [ ]:
# Here we are primarly interested in evaluating the response quality since we are using the same retriver in both pipelines
# We will use LLM metrics to evaluate the response quality.

In [14]:
from scripts.response_metrics import LLM_METRICS

response_evaluations = weave.Evaluation(
    name="Response_Evaluation",
    dataset=eval_dataset,
    scorers=LLM_METRICS,
    preprocess_model_input=lambda x: {"query": x["question"]},
)

baseline_response_scores = asyncio.run(
    response_evaluations.evaluate(simple_rag_pipeline)
)

query_enhanced_response_scores = asyncio.run(
    response_evaluations.evaluate(query_enhanced_rag_pipeline)
)

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

...

Evaluated 17 of 20 examples

Evaluated 18 of 20 examples

Evaluated 19 of 20 examples

Evaluated 20 of 20 examples

Evaluation summary
{
    'llm_response_scorer': {'score': {'mean': 0.7}, 'correct': {'true_count': 1, 'true_fraction': 0.05}},
    'model_latency': {'mean': 21.39699239730835}
}

![compare_retriever_responses](../images/04_compare_query_enhanced_responses.png)